In [1]:
import json

with open("vendor-purchased-2019_tweets.json", "r") as f:
    dataset = json.load(f)
print(len(dataset))

features = {"friends/follower_ratio" : [], "is_bot_in_name":[]}
labels = []
for idx, instance in enumerate(dataset):
    UTC = instance["created_at"]
    user = instance["user"]
    friends = user["friends_count"]
    followers = user["followers_count"]
    
    #Get "friends/follower_ratio" feature
    if followers == 0 :
        ratio = 0
    else:
        ratio = friends/followers
    features["friends/follower_ratio"].append(ratio)
    
    #Get "is_bot_in_name" feature
    if "bot" in user["screen_name"]:
        is_name = 1
    else:
        is_name = 0
    features["is_bot_in_name"].append(is_name)
    labels.append(idx)

1088


In [2]:
def divide_dataset(X, y) :
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    return (X_train, X_test, y_train, y_test)

X = []
for point in zip(features["friends/follower_ratio"], features["is_bot_in_name"]):    
    X.append(point)
y = labels

X_train, X_test, y_train, y_test = divide_dataset(X, y)

In [3]:
import time
import datetime
def train(classifier, name, param_grid=None) :
    start_time = time.time()
    if param_grid == None :
        classifier.fit(X_train, y_train)
        results[name] = dict(model=classifier)
    else :
        grid = GridSearchCV(classifier, param_grid, cv=10, scoring='accuracy', n_jobs=2) # Do a 10-fold cross validation
        grid.fit(X, y) # fit the grid with data
        results[name] = dict(grid=grid, model=classifier)
    #total_time = datetime.datetime.fromtimestamp(time.time() - start_time)
    total_time = datetime.timedelta(seconds=time.time() - start_time)
    print("Training time : " + str(total_time))#.strftime('%H:%M:%S'))

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

In [6]:
import numpy as np

name = "SVM - SVC"
classifier = svm.SVC()
C_range = np.logspace(-2, 10, 13)
#print(C_range)
gamma_range = np.logspace(-9, 3, 13)
#print(gamma_range)
param_grid = dict(gamma=gamma_range, C=C_range)
train(classifier, name, param_grid)

ValueError: n_splits=10 cannot be greater than the number of members in each class.

In [7]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

print(clf.predict([[0.5, 0]]))

[971]
